# Autogen을 활용한 Agentic RAG

이 노트북은 향상된 평가 기능을 갖춘 Autogen 에이전트를 사용하여 검색 증강 생성(RAG)을 구현하는 방법을 보여줍니다.


SQLite 버전 수정  
다음 오류가 발생할 경우:  
```
RuntimeError: Your system has an unsupported version of sqlite3. Chroma requires sqlite3 >= 3.35.0
```  

노트북 시작 부분에서 이 코드 블록의 주석을 해제하세요:  


In [2]:
# %pip install pysqlite3-binary
# __import__('pysqlite3')
# import sys
# sys.modules['sqlite3'] = sys.modules.pop('pysqlite3')

In [3]:
import os
import time
import asyncio
from typing import List, Dict
from dotenv import load_dotenv

from autogen_agentchat.agents import AssistantAgent
from autogen_core import CancellationToken
from autogen_agentchat.messages import TextMessage
from azure.core.credentials import AzureKeyCredential
from autogen_ext.models.azure import AzureAIChatCompletionClient

import chromadb

load_dotenv()

True

## 클라이언트 생성

먼저, Azure AI Chat Completion 클라이언트를 초기화합니다. 이 클라이언트는 사용자 질문에 대한 응답을 생성하기 위해 Azure OpenAI 서비스를 활용하는 데 사용됩니다.


In [4]:
client = AzureAIChatCompletionClient(
    model="gpt-4o-mini",
    endpoint="https://models.inference.ai.azure.com",
    credential=AzureKeyCredential(os.getenv("GITHUB_TOKEN")),
    model_info={
        "json_output": True,
        "function_calling": True,
        "vision": True,
        "family": "unknown",
    },
)

## 벡터 데이터베이스 초기화

ChromaDB를 영구 저장소와 함께 초기화하고, 향상된 샘플 문서를 추가합니다. ChromaDB는 문서를 저장하고 검색하여 정확한 응답을 생성하는 데 필요한 컨텍스트를 제공합니다.


In [5]:
# Initialize ChromaDB with persistent storage
chroma_client = chromadb.PersistentClient(path="./chroma_db")
collection = chroma_client.create_collection(
    name="travel_documents",
    metadata={"description": "travel_service"},
    get_or_create=True
)

# Enhanced sample documents
documents = [
    "Contoso Travel offers luxury vacation packages to exotic destinations worldwide.",
    "Our premium travel services include personalized itinerary planning and 24/7 concierge support.",
    "Contoso's travel insurance covers medical emergencies, trip cancellations, and lost baggage.",
    "Popular destinations include the Maldives, Swiss Alps, and African safaris.",
    "Contoso Travel provides exclusive access to boutique hotels and private guided tours."
]

# Add documents with metadata
collection.add(
    documents=documents,
    ids=[f"doc_{i}" for i in range(len(documents))],
    metadatas=[{"source": "training", "type": "explanation"} for _ in documents]
)

## 컨텍스트 제공자 구현

`ContextProvider` 클래스는 여러 소스에서 컨텍스트를 검색하고 통합하는 역할을 합니다:

1. **벡터 데이터베이스 검색**: ChromaDB를 사용하여 여행 문서에 대한 의미론적 검색 수행
2. **날씨 정보**: 주요 도시의 시뮬레이션된 날씨 데이터베이스를 유지
3. **통합 컨텍스트**: 문서 데이터와 날씨 데이터를 결합하여 포괄적인 컨텍스트 제공

주요 메서드:
- `get_retrieval_context()`: 쿼리에 기반하여 관련 문서를 검색
- `get_weather_data()`: 지정된 위치에 대한 날씨 정보를 제공
- `get_unified_context()`: 문서와 날씨 컨텍스트를 결합하여 향상된 응답 제공


In [6]:
class ContextProvider:
    def __init__(self, collection):
        self.collection = collection
        # Simulated weather database
        self.weather_database = {
            "new york": {"temperature": 72, "condition": "Partly Cloudy", "humidity": 65, "wind": "10 mph"},
            "london": {"temperature": 60, "condition": "Rainy", "humidity": 80, "wind": "15 mph"},
            "tokyo": {"temperature": 75, "condition": "Sunny", "humidity": 50, "wind": "5 mph"},
            "sydney": {"temperature": 80, "condition": "Clear", "humidity": 45, "wind": "12 mph"},
            "paris": {"temperature": 68, "condition": "Cloudy", "humidity": 70, "wind": "8 mph"},
        }
    
    def get_retrieval_context(self, query: str) -> str:
        """Retrieves relevant documents from vector database based on query."""
        results = self.collection.query(
            query_texts=[query],
            include=["documents", "metadatas"],
            n_results=2
        )
        context_strings = []
        if results and results.get("documents") and len(results["documents"][0]) > 0:
            for doc, meta in zip(results["documents"][0], results["metadatas"][0]):
                context_strings.append(f"Document: {doc}\nMetadata: {meta}")
        return "\n\n".join(context_strings) if context_strings else "No relevant documents found"
    
    def get_weather_data(self, location: str) -> str:
        """Simulates retrieving weather data for a given location."""
        if not location:
            return ""
            
        location_key = location.lower()
        if location_key in self.weather_database:
            data = self.weather_database[location_key]
            return f"Weather for {location.title()}:\n" \
                   f"Temperature: {data['temperature']}°F\n" \
                   f"Condition: {data['condition']}\n" \
                   f"Humidity: {data['humidity']}%\n" \
                   f"Wind: {data['wind']}"
        else:
            return f"No weather data available for {location}."
    
    def get_unified_context(self, query: str, location: str = None) -> str:
        """Returns a unified context combining both document retrieval and weather data."""
        retrieval_context = self.get_retrieval_context(query)
        
        weather_context = ""
        if location:
            weather_context = self.get_weather_data(location)
            weather_intro = f"\nWeather Information for {location}:\n"
        else:
            weather_intro = ""
        
        return f"Retrieved Context:\n{retrieval_context}\n\n{weather_intro}{weather_context}"

## 에이전트 구성

우리는 검색 에이전트와 보조 에이전트를 구성합니다. 검색 에이전트는 의미론적 검색을 사용하여 관련 정보를 찾는 데 특화되어 있으며, 보조 에이전트는 검색된 정보를 바탕으로 상세한 응답을 생성합니다.


In [7]:
# Create agents with enhanced capabilities
assistant = AssistantAgent(
    name="assistant",
    model_client=client,
    system_message=(
        "You are a helpful AI assistant that provides answers using ONLY the provided context. "
        "Do NOT include any external information. Base your answer entirely on the context given below."
    ),
)

## RAG를 사용한 쿼리 처리

우리는 `ask_rag` 함수를 정의하여 쿼리를 어시스턴트에게 보내고, 응답을 처리하며, 이를 평가합니다. 이 함수는 어시스턴트와의 상호작용을 처리하고, 평가자를 사용하여 응답의 품질을 측정합니다.


In [8]:
async def ask_rag_agent(query: str, context_provider: ContextProvider, location: str = None):
    """
    Sends a query to the assistant agent with context from the provider.
    
    Args:
        query: The user's question
        context_provider: The context provider instance
        location: Optional location for weather queries
    """
    try:
        # Get unified context
        context = context_provider.get_unified_context(query, location)
        
        # Augment the query with context
        augmented_query = (
            f"{context}\n\n"
            f"User Query: {query}\n\n"
            "Based ONLY on the above context, please provide a helpful answer."
        )

        # Send the augmented query to the assistant
        start_time = time.time()
        response = await assistant.on_messages(
            [TextMessage(content=augmented_query, source="user")],
            cancellation_token=CancellationToken(),
        )
        processing_time = time.time() - start_time
        
        return {
            'query': query,
            'response': response.chat_message.content,
            'processing_time': processing_time,
            'location': location
        }
    except Exception as e:
        print(f"Error processing query: {e}")
        return None

# 사용 예시

평가기를 초기화하고 처리 및 평가하려는 쿼리를 정의합니다.


In [ ]:
async def main():
    # Initialize context provider
    context_provider = ContextProvider(collection)
    
    # Example queries
    queries = [
        {"query": "What does Contoso's travel insurance cover?"},
        {"query": "What's the weather like in London?", "location": "london"},
        {"query": "What luxury destinations does Contoso offer and what's the weather in Paris?", "location": "paris"},
    ]
    
    print("=== Autogen RAG Demo ===")
    for query_data in queries:
        query = query_data["query"]
        location = query_data.get("location")
        
        print(f"\n\nQuery: {query}")
        if location:
            print(f"Location: {location}")
        
        # Show the context being used
        context = context_provider.get_unified_context(query, location)
        print("\n--- Context Used ---")
        print(context)
        print("-------------------")
        
        # Get response from the agent
        result = await ask_rag_agent(query, context_provider, location)
        if result:
            print(f"\nResponse: {result['response']}")
        print("\n" + "="*50)

## 스크립트 실행하기

스크립트가 인터랙티브 환경에서 실행 중인지 또는 일반 스크립트로 실행 중인지 확인한 후, 이에 따라 main 함수를 실행합니다.


In [12]:
if __name__ == "__main__":
    if asyncio.get_event_loop().is_running():
        await main()
    else:
        asyncio.run(main())

=== Autogen RAG Demo ===


Query: What does Contoso's travel insurance cover?

--- Context Used ---
Retrieved Context:
Document: Contoso's travel insurance covers medical emergencies, trip cancellations, and lost baggage.
Metadata: {'source': 'training', 'type': 'explanation'}

Document: Contoso Travel offers luxury vacation packages to exotic destinations worldwide.
Metadata: {'source': 'training', 'type': 'explanation'}


-------------------

Response: Contoso's travel insurance covers medical emergencies, trip cancellations, and lost baggage.



Query: What's the weather like in London?
Location: london

--- Context Used ---
Retrieved Context:
Document: Popular destinations include the Maldives, Swiss Alps, and African safaris.
Metadata: {'source': 'training', 'type': 'explanation'}

Document: Contoso Travel offers luxury vacation packages to exotic destinations worldwide.
Metadata: {'source': 'training', 'type': 'explanation'}


Weather Information for london:
Weather for London:
T


---

**면책 조항**:  
이 문서는 AI 번역 서비스 [Co-op Translator](https://github.com/Azure/co-op-translator)를 사용하여 번역되었습니다. 정확성을 위해 최선을 다하고 있으나, 자동 번역에는 오류나 부정확성이 포함될 수 있습니다. 원본 문서의 원어 버전이 권위 있는 출처로 간주되어야 합니다. 중요한 정보의 경우, 전문적인 인간 번역을 권장합니다. 이 번역 사용으로 인해 발생하는 오해나 잘못된 해석에 대해 당사는 책임을 지지 않습니다.
